<a href="https://colab.research.google.com/github/DavidVollendroff/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-10-30 19:28:43--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.72.218
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.72.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  79.9MB/s    in 2.5s    

2019-10-30 19:28:46 (79.9 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [4]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [8]:
##### YOUR CODE HERE #####
import pandas as pd

aisles = pd.read_csv('aisles.csv')
print(aisles.columns)
departments = pd.read_csv('departments.csv')
print(departments.columns)
order_products__prior = pd.read_csv('order_products__prior.csv')
print(order_products__prior.columns)
order_products__train = pd.read_csv('order_products__train.csv')
print(order_products__train.columns)
orders = pd.read_csv('orders.csv')
print(orders.columns)
products = pd.read_csv('products.csv')
print(products.columns)

Index(['aisle_id', 'aisle'], dtype='object')
Index(['department_id', 'department'], dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')
Index(['product_id', 'product_name', 'aisle_id', 'department_id'], dtype='object')


In [20]:
# Firstly it seem appropriate to concatenate __prior and __train
order_products = pd.concat((order_products__prior, order_products__train))
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [24]:
# I need product_name and order_id to determine the number of instances of
# product orders
merge1 = pd.merge(products[['product_name', 'product_id']], order_products[['product_id', 'order_id']], on='product_id')
merge1.head()

,product_name,product_id,order_id
0,Chocolate Sandwich Cookies,1,1107
1,Chocolate Sandwich Cookies,1,5319
2,Chocolate Sandwich Cookies,1,7540
3,Chocolate Sandwich Cookies,1,9228
4,Chocolate Sandwich Cookies,1,9273


In [37]:
value_counts = merge1['product_name'].value_counts()[:10]
value_counts['Banana']
value_dict = {}
for item in value_counts.index:
  value_dict[item] = value_counts[item]
value_dict
# I chose to make a dictionary for the practice, but the value_counts variable
# is already in rank order in case that is the preferred item.

{'Bag of Organic Bananas': 394930,
 'Banana': 491291,
 'Large Lemon': 160792,
 'Limes': 146660,
 'Organic Avocado': 184224,
 'Organic Baby Spinach': 251705,
 'Organic Hass Avocado': 220877,
 'Organic Strawberries': 275577,
 'Organic Whole Milk': 142813,
 'Strawberries': 149445}

## Reshape Data Section

- Replicate the lesson code
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

## Replicating the lesson code

In [0]:
# I followed along with the lesson so much of this maybe copied and pasted for
# the sake of making it to the stretch goals

In [40]:
# So we know what the goal is
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

# Because I've already created the dataframes and printed their column names I will omit the exploratory process here. It has already been done in this notebook

In [0]:
# from order_products I need
order_product_columns = ['order_id', 'product_id', 'add_to_cart_order']

# from orders I need
order_columns = ['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day']

# from products I need
products_columns = ['product_id', 'product_name']

In [45]:
# making subset of orders
condition = (orders['order_number'] <=2) & (orders['user_id'] == 1)

orders_subset = orders.loc[condition, order_columns]
orders_subset.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day
0,2539329,1,1,2,8
1,2398795,1,2,3,7


In [0]:
# and to make an order_products subset
condition2 = (order_products['order_id'] == 2539329) | (order_products['order_id'] == 2398795)

order_products_subset = order_products.loc[condition2, order_product_columns]
order_products_subset.head(15)

In [48]:
# now to merge the two subsets
merged_subsets = pd.merge(orders_subset, order_products_subset,
                          on='order_id', how='left')
merged_subsets.head(15)

,order_id,user_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order
0,2539329,1,1,2,8,196,1
1,2539329,1,1,2,8,14084,2
2,2539329,1,1,2,8,12427,3
3,2539329,1,1,2,8,26088,4
4,2539329,1,1,2,8,26405,5
5,2398795,1,2,3,7,196,1
6,2398795,1,2,3,7,10258,2
7,2398795,1,2,3,7,12427,3
8,2398795,1,2,3,7,13176,4
9,2398795,1,2,3,7,26088,5


In [49]:
display(example)

In [51]:
# now we are missing only product names and formating column headers
final = pd.merge(merged_subsets,
                 products[products_columns],
                 on='product_id',
                 how='left') # merge left in case of missing name product_name
final.columns = [column.replace('_', ' ') for column in final]
final.head(15)

,order id,user id,order number,order dow,order hour of day,product id,add to cart order,product name
0,2539329,1,1,2,8,196,1,Soda
1,2539329,1,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
2,2539329,1,1,2,8,12427,3,Original Beef Jerky
3,2539329,1,1,2,8,26088,4,Aged White Cheddar Popcorn
4,2539329,1,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
5,2398795,1,2,3,7,196,1,Soda
6,2398795,1,2,3,7,10258,2,Pistachios
7,2398795,1,2,3,7,12427,3,Original Beef Jerky
8,2398795,1,2,3,7,13176,4,Bag of Organic Bananas
9,2398795,1,2,3,7,26088,5,Aged White Cheddar Popcorn


# Reshaping datasets

In [0]:
# Getting things started with the canonical example
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

In [64]:
table1

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


Our first table

In [65]:
table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


The transposed table

In [66]:
# Transforming table1 to tidy
table1 = table1.reset_index()
table1

,index,treatmenta,treatmentb
0,John Smith,NaN,2
1,Jane Doe,16.0,11
2,Mary Johnson,3.0,1


In [67]:
# using pd.melt to creat a tidy-formatted dataframe
tidy1 = table1.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])
tidy1

,index,variable,value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [68]:
# renaming columns
tidy1 = tidy1.rename(columns={
    'index': 'name', 
    'variable': 'trt', 
    'value': 'result'
})
tidy1 # now looks like table 3

,name,trt,result
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [85]:
# tidying table2
table2 = table2.reset_index()
table2


,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [98]:
# transforming via melt
tidy2 = table2.melt(id_vars='index', value_vars=['John Smith', 'Jane Doe', 'Mary Johnson'],)
tidy2

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [100]:
# reordering
tidy2 = tidy2[['variable', 'index', 'value']]
tidy2

,variable,index,value
0,John Smith,treatmenta,NaN
1,John Smith,treatmentb,2.0
2,Jane Doe,treatmenta,16.0
3,Jane Doe,treatmentb,11.0
4,Mary Johnson,treatmenta,3.0
5,Mary Johnson,treatmentb,1.0


,name,trt,result
0,John Smith,treatmenta,NaN
1,John Smith,treatmentb,2.0
2,Jane Doe,treatmenta,16.0
3,Jane Doe,treatmentb,11.0
4,Mary Johnson,treatmenta,3.0
5,Mary Johnson,treatmentb,1.0


In [106]:
# tidy to table1 using pivot_table
wide = tidy1.pivot_table(index='name', columns='trt', values='result')
wide

trt,treatmenta,treatmentb
name,,
Jane Doe,16.0,11.0
John Smith,NaN,2.0
Mary Johnson,3.0,1.0


In [129]:
# from tidy to table 2
new_table2 = tidy1.pivot_table(index='trt', columns='name', values='result')
new_table2

name,Jane Doe,John Smith,Mary Johnson
trt,,,
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


In [130]:
# renaming
new_table2 = new_table2.rename(columns={'variable':'name', 'index':'trt', 'value':'result'})
new_table2

name,Jane Doe,John Smith,Mary Johnson
trt,,,
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


In [131]:
# some bits of formatting
new_table2.index.name = None
new_table2.columns.name = None
new_table2

,Jane Doe,John Smith,Mary Johnson
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
flights = sns.load_dataset('flights')

In [114]:
##### YOUR CODE HERE #####
flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [115]:
flights_pivot = flights.pivot_table(index='year', columns='month', values='passengers')
flights_pivot

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####